<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #11: Concatenate files from AWS</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

## Install kso_data_management and its requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso_data_management and its requirements</font> { vertical-output: true }

from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/requirements.txt) -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso-data-management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False

    # Install requirements
    %pip install -q --no-warn-script-location --upgrade pip
    %pip install -qr ../requirements.txt -qr ../kso_utils/requirements.txt

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    !jupyter nbextension install --user --py ipysheet
    !jupyter nbextension enable --user --py ipysheet

    clear_output()
    print("Running locally... you're good to go!")

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.append("..")

# Enable third party widgets
from google.colab import output

output.enable_custom_widget_manager()

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t11_utils as t11

print("Packages loaded successfully")

## Choose your project

In [ ]:
project_name = t_utils.choose_project()

## Initiate sql and get server or local storage details

In [ ]:
# Save project name
project = p_utils.find_project(project_name=project_name.value)
# Initiate db
db_info_dict = t_utils.initiate_db(project)

## Get info from go pro movies

In [ ]:
missing_from_server, missing_from_csv = t11.check_movies_from_server(
    db_info_dict, project
)

## Select deployment recorded on the same date

In [ ]:
deployment_selected = t11.select_deployment(missing_from_csv)

## Select date of recording

In [ ]:
event_date = t11.select_eventdate()

## Concatenate the movies

In [ ]:
t11.update_new_deployments(deployment_selected, db_info_dict, event_date)

In [ ]:
# END